In [ ]:
try:
    import resnest
except ModuleNotFoundError:
    !pip install -q "../input/resnest-package/resnest-0.0.6b20200701/resnest"

In [ ]:
import re
import cv2
import time
import torch
import numpy as np
import pandas as pd
import librosa as lb
from torch import nn
import soundfile as sf
from pathlib import Path
from tqdm.notebook import tqdm
from collections import defaultdict
from resnest.torch import resnest50, resnest101
from  torch.utils.data import Dataset, DataLoader

# Configs

In [ ]:
NUM_CLASSES = 397
SR = 32_000
DURATION = 5

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

TEST_AUDIO_ROOT = Path("../input/birdclef-2021/test_soundscapes")
SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
TARGET_PATH = None
    
if not len(list(TEST_AUDIO_ROOT.glob("*.ogg"))):
    TEST_AUDIO_ROOT = Path("../input/birdclef-2021/train_soundscapes")
    SAMPLE_SUB_PATH = None
    TARGET_PATH = Path("../input/birdclef-2021/train_soundscape_labels.csv")

# Functions

In [ ]:
class MelSpecComputer:
    def __init__(self, sr, n_mels, fmin, fmax, **kwargs):
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax
        kwargs["n_fft"] = kwargs.get("n_fft", self.sr//10)
        kwargs["hop_length"] = kwargs.get("hop_length", self.sr//(10*4))
        self.kwargs = kwargs

    def __call__(self, y):

        melspec = lb.feature.melspectrogram(
            y, sr=self.sr, n_mels=self.n_mels, fmin=self.fmin, fmax=self.fmax, **self.kwargs,
        )

        melspec = lb.power_to_db(melspec).astype(np.float32)
        return melspec
    
def mono_to_color(X, eps=1e-6, mean=None, std=None):
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)
    
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)

    return V

def crop_or_pad(y, length):
    if len(y) < length:
        y = np.concatenate([y, length - np.zeros(len(y))])
    elif len(y) > length:
        y = y[:length]
    return y

class BirdCLEFDataset(Dataset):
    def __init__(self, data, sr=SR, n_mels=128, fmin=0, fmax=None, duration=DURATION, step=None, res_type="kaiser_fast", resample=True):
        
        self.data = data
        
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax or self.sr//2

        self.duration = duration
        self.audio_length = self.duration*self.sr
        self.step = step or self.audio_length
        
        self.res_type = res_type
        self.resample = resample

        self.mel_spec_computer = MelSpecComputer(sr=self.sr, n_mels=self.n_mels, fmin=self.fmin,
                                                 fmax=self.fmax)
    def __len__(self):
        return len(self.data)
    
    @staticmethod
    def normalize(image):
        image = image.astype("float32", copy=False) / 255.0
        image = np.stack([image, image, image])
        return image
    
    def audio_to_image(self, audio):
        melspec = self.mel_spec_computer(audio) 
        image = mono_to_color(melspec)
        image = self.normalize(image)
        return image

    def read_file(self, filepath):
        audio, orig_sr = sf.read(filepath, dtype="float32")

        if self.resample and orig_sr != self.sr:
            audio = lb.resample(audio, orig_sr, self.sr, res_type=self.res_type)
          
        audios = []
        for i in range(self.audio_length, len(audio) + self.step, self.step):
            start = max(0, i - self.audio_length)
            end = start + self.audio_length
            audios.append(audio[start:end])
            
        if len(audios[-1]) < self.audio_length:
            audios = audios[:-1]
            
        images = [self.audio_to_image(audio) for audio in audios]
        images = np.stack(images)
        
        return images
    
        
    def __getitem__(self, idx):
        return self.read_file(self.data.loc[idx, "filepath"])
    
def load_net(checkpoint_path, num_classes=NUM_CLASSES):
    
    if 'resnest50' in checkpoint_path:
        net = resnest50(pretrained=False)
    else:
        net = resnest101(pretrained=False)

    net.fc = nn.Linear(net.fc.in_features, num_classes)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)
    for key in list(d.keys()):
        d[key.replace("model.", "")] = d.pop(key)
    net.load_state_dict(d)
    net = net.to(DEVICE)
    net = net.eval()
    return net

@torch.no_grad()
def get_thresh_preds(out, thresh=None):
    thresh = thresh or THRESH
    o = (-out).argsort(1)
    npreds = (out > thresh).sum(1)
    preds = []
    for oo, npred in zip(o, npreds):
        preds.append(oo[:npred].cpu().numpy().tolist())
    return preds

def get_bird_names(preds):
    bird_names = []
    for pred in preds:
        if not pred:
            bird_names.append("nocall")
        else:
            bird_names.append(" ".join([INV_LABEL_IDS[bird_id] for bird_id in pred]))
    return bird_names


def predict(nets, test_data, names=True):
    preds = []
    with torch.no_grad():
        for idx in  tqdm(list(range(len(test_data)))):
            xb = torch.from_numpy(test_data[idx]).to(DEVICE)
            pred = 0.
            for net in nets:
                o = net(xb)
                o = torch.sigmoid(o)

                pred += o

            pred /= len(nets)
            
            if names:
                pred = get_bird_names(get_thresh_preds(pred))

            preds.append(pred)
    return preds


def preds_as_df(data, preds):
    sub = {
        "row_id": [],
        "birds": [],
    }
    
    for row, pred in zip(data.itertuples(False), preds):
        row_id = [f"{row.id}_{row.site}_{5*i}" for i in range(1, len(pred)+1)]
        sub["birds"] += pred
        sub["row_id"] += row_id
        
    sub = pd.DataFrame(sub)
    
    if SAMPLE_SUB_PATH:
        sample_sub = pd.read_csv(SAMPLE_SUB_PATH, usecols=["row_id"])
        sub = sample_sub.merge(sub, on="row_id", how="left")
        sub["birds"] = sub["birds"].fillna("nocall")
    return sub

def get_metrics(s_true, s_pred):
    s_true = set(s_true.split())
    s_pred = set(s_pred.split())
    n, n_true, n_pred = len(s_true.intersection(s_pred)), len(s_true), len(s_pred)
    
    prec = n/n_pred
    rec = n/n_true
    f1 = 2*prec*rec/(prec + rec) if prec + rec else 0
    
    return {"f1": f1, "prec": prec, "rec": rec, "n_true": n_true, "n_pred": n_pred, "n": n}

def def_value():
    return 1.0

In [ ]:
data = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in Path(TEST_AUDIO_ROOT).glob("*.ogg")],
    columns = ["filename", "id", "site", "date", "filepath"]
)
print(data.shape)
data.head()

In [ ]:
df_train = pd.read_csv("../input/birdclef-2021/train_metadata.csv")

LABEL_IDS = {label: label_id for label_id,label in enumerate(sorted(df_train["primary_label"].unique()))}
INV_LABEL_IDS = {val: key for key,val in LABEL_IDS.items()}

# Inference

In [ ]:
test_data = BirdCLEFDataset(data=data)
len(test_data), test_data[0].shape

In [ ]:
raw_preds = {}

# MODEL 1


In [ ]:
MODEL1 = [f'../input/kkiller-birdclef-models-public/birdclef_resnest50_fold0_epoch_10_f1_val_06471_20210417161101.pth']

for m in MODEL1:
    checkpoint_paths = [Path(m)]

nets = [load_net(checkpoint_path.as_posix()) for checkpoint_path in checkpoint_paths]
pred_probas = predict(nets, test_data, names=False)
raw_preds['MODEL1'] = pred_probas

# MODEL 2

In [ ]:
MODEL2 = [
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v11/resnest101_sr32000_d7_v11/birdclef_resnest101_fold0_epoch_16_f1_val_07591_20210519181210.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v11/resnest101_sr32000_d7_v11/birdclef_resnest101_fold1_epoch_13_f1_val_07365_20210519220652.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v11/resnest101_sr32000_d7_v11/birdclef_resnest101_fold2_epoch_14_f1_val_07521_20210520024418.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v11/resnest101_sr32000_d7_v11/birdclef_resnest101_fold3_epoch_15_f1_val_07583_20210520072224.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v11/resnest101_sr32000_d7_v11/birdclef_resnest101_fold4_epoch_10_f1_val_07331_20210520105725.pth'),
  
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v8/resnest101_sr32000_d7_v8/birdclef_resnest101_fold0_epoch_16_f1_val_07734_20210518233713.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v8/resnest101_sr32000_d7_v8/birdclef_resnest101_fold1_epoch_16_f1_val_07675_20210519030243.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v8/resnest101_sr32000_d7_v8/birdclef_resnest101_fold2_epoch_12_f1_val_07466_20210519054713.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v8/resnest101_sr32000_d7_v8/birdclef_resnest101_fold3_epoch_13_f1_val_07424_20210519092316.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v8/resnest101_sr32000_d7_v8/birdclef_resnest101_fold4_epoch_18_f1_val_07755_20210519134027.pth'), 
]

checkpoint_paths = MODEL2

nets = [load_net(checkpoint_path.as_posix()) for checkpoint_path in checkpoint_paths]
pred_probas = predict(nets, test_data, names=False)
raw_preds['MODEL2'] = pred_probas

# MODEL 3

In [ ]:
MODEL3 = [
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v27/resnest101_sr32000_d7_v27/birdclef_resnest101_fold1_epoch_09_f1_val_05461_20210526015608.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v27/resnest101_sr32000_d7_v27/birdclef_resnest101_fold1_epoch_11_f1_val_05736_20210526020742.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v27/resnest101_sr32000_d7_v27/birdclef_resnest101_fold2_epoch_13_f1_val_05963_20210526045546.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v27/resnest101_sr32000_d7_v27/birdclef_resnest101_fold3_epoch_11_f1_val_05636_20210526072043.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v27/resnest101_sr32000_d7_v27/birdclef_resnest101_fold4_epoch_14_f1_val_05889_20210526101445.pth'),
    
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v26/resnest101_sr32000_d7_v26/birdclef_resnest101_fold0_epoch_14_f1_val_06002_20210525035058.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v26/resnest101_sr32000_d7_v26/birdclef_resnest101_fold1_epoch_11_f1_val_05715_20210525075316.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v26/resnest101_sr32000_d7_v26/birdclef_resnest101_fold2_epoch_11_f1_val_05670_20210525122558.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v26/resnest101_sr32000_d7_v26/birdclef_resnest101_fold3_epoch_11_f1_val_05598_20210525165840.pth'),
]

checkpoint_paths = MODEL3

nets = [load_net(checkpoint_path.as_posix()) for checkpoint_path in checkpoint_paths]
pred_probas = predict(nets, test_data, names=False)
raw_preds['MODEL3'] = pred_probas

# MODEL 4

In [ ]:
MODEL4 = [
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v32/resnest101_sr32000_d7_v32/birdclef_resnest101_fold0_epoch_02_f1_val_05618_20210530142121.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v32/resnest101_sr32000_d7_v32/birdclef_resnest101_fold1_epoch_02_f1_val_05536_20210530181637.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v32/resnest101_sr32000_d7_v32/birdclef_resnest101_fold2_epoch_02_f1_val_05802_20210530220945.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v32/resnest101_sr32000_d7_v32/birdclef_resnest101_fold3_epoch_02_f1_val_05935_20210531020315.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v32/resnest101_sr32000_d7_v32/birdclef_resnest101_fold4_epoch_02_f1_val_05595_20210531055634.pth'),
    
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v32/resnest101_sr32000_d7_v32/birdclef_resnest101_fold0_epoch_05_f1_val_07128_20210530150823.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v32/resnest101_sr32000_d7_v32/birdclef_resnest101_fold1_epoch_05_f1_val_07131_20210530190322.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v32/resnest101_sr32000_d7_v32/birdclef_resnest101_fold2_epoch_05_f1_val_07029_20210530225631.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v32/resnest101_sr32000_d7_v32/birdclef_resnest101_fold3_epoch_05_f1_val_07148_20210531024952.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v32/resnest101_sr32000_d7_v32/birdclef_resnest101_fold4_epoch_05_f1_val_07087_20210531064317.pth'),
]


checkpoint_paths = MODEL4

nets = [load_net(checkpoint_path.as_posix()) for checkpoint_path in checkpoint_paths]
pred_probas = predict(nets, test_data, names=False)
raw_preds['MODEL4'] = pred_probas

# MODEL 5

In [ ]:
MODEL5 = [
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v28/resnest101_sr32000_d7_v28/birdclef_resnest101_fold0_epoch_12_f1_val_05806_20210526212606.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v28/resnest101_sr32000_d7_v28/birdclef_resnest101_fold1_epoch_12_f1_val_05777_20210526232211.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v28/resnest101_sr32000_d7_v28/birdclef_resnest101_fold2_epoch_09_f1_val_05546_20210527010045.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v28/resnest101_sr32000_d7_v28/birdclef_resnest101_fold3_epoch_12_f1_val_05634_20210527031422.pth'),
    Path(f'../input/birdclef-models/resnest101_sr32000_d7_v28/resnest101_sr32000_d7_v28/birdclef_resnest101_fold4_epoch_09_f1_val_05711_20210527045319.pth'),
]

checkpoint_paths = MODEL5

nets = [load_net(checkpoint_path.as_posix()) for checkpoint_path in checkpoint_paths]
pred_probas = predict(nets, test_data, names=False)
raw_preds['MODEL5'] = pred_probas

# MODEL 6

In [ ]:
MODEL6 = [
    Path(f'../input/birdclef-models/resnest50_sr32000_d7_v9/resnest50_sr32000_d7_v9/birdclef_resnest50_fold0_epoch_22_f1_val_07582_20210519030656.pth'),
    Path(f'../input/birdclef-models/resnest50_sr32000_d7_v9/resnest50_sr32000_d7_v9/birdclef_resnest50_fold1_epoch_29_f1_val_07637_20210519053244.pth'),
    Path(f'../input/birdclef-models/resnest50_sr32000_d7_v9/resnest50_sr32000_d7_v9/birdclef_resnest50_fold2_epoch_22_f1_val_07535_20210519070947.pth'),
    Path(f'../input/birdclef-models/resnest50_sr32000_d7_v9/resnest50_sr32000_d7_v9/birdclef_resnest50_fold3_epoch_23_f1_val_07607_20210519091446.pth'),
    Path(f'../input/birdclef-models/resnest50_sr32000_d7_v9/resnest50_sr32000_d7_v9/birdclef_resnest50_fold4_epoch_26_f1_val_07666_20210519112640.pth'),
    
    Path(f'../input/birdclef-models/resnest50_sr32000_d7_v3/resnest50_sr32000_d7_v3/birdclef_resnest50_fold2_epoch_22_f1_val_07690_20210518082321.pth'),
]

checkpoint_paths = MODEL6

nets = [load_net(checkpoint_path.as_posix()) for checkpoint_path in checkpoint_paths]
pred_probas = predict(nets, test_data, names=False)
raw_preds['MODEL6'] = pred_probas

In [ ]:
squared_models = {}

# COMBINING MODELS

In [ ]:
consider_models = ['MODEL1', 'MODEL2', 'MODEL3', 'MODEL4', 'MODEL5', 'MODEL6']

squared_preds = {}
for model in consider_models:
    
    res_t = []
    for file in raw_preds[model]:
        res_t.append(torch.square(file))
    squared_preds[model] = res_t

mean_preds = []

for t in range(len(squared_preds[consider_models[0]])):
    
    squared_list = []
    for model in squared_preds:
        squared_list.append(squared_preds[model][t])

        
    mean_preds.append(torch.mean(torch.stack(squared_list),0))
    
final_preds = []

for t in mean_preds:
    final_preds.append(torch.sqrt(t))
    
    
squared_models['_'.join(consider_models)] = final_preds

In [ ]:
th = 0.165
preds = [get_bird_names(get_thresh_preds(pred, thresh=th)) for pred in final_preds]
submission_df = preds_as_df(data, preds)
print(submission_df.shape)
submission_df

In [ ]:
submission_df.to_csv("submission.csv", index=False)

# Small validation

In [ ]:
if TARGET_PATH:
    sub_target = pd.read_csv(TARGET_PATH)
    sub_target = sub_target.merge(submission_df, how="left", on="row_id")
    
    print(sub_target["birds_x"].notnull().sum(), sub_target["birds_x"].notnull().sum())
    assert sub_target["birds_x"].notnull().all()
    assert sub_target["birds_y"].notnull().all()
    
    df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target.birds_x, sub_target.birds_y)])
    
    print(df_metrics.mean())

In [ ]:
df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target[sub_target.birds_y != "nocall"].birds_x, sub_target[sub_target.birds_y != "nocall"].birds_y)])
print(df_metrics.mean())

display(sub_target[sub_target.birds_y != "nocall"])

In [ ]:
df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target[sub_target.birds_x != "nocall"].birds_x, sub_target[sub_target.birds_x != "nocall"].birds_y)])
print(df_metrics.mean())


display(sub_target[sub_target.birds_x != "nocall"])

In [ ]:
df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target[sub_target.birds_y == "nocall"].birds_x, sub_target[sub_target.birds_y == "nocall"].birds_y)])
print(df_metrics.mean())

display(sub_target[sub_target.birds_y == "nocall"])

In [ ]:
df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target[sub_target.birds_x == "nocall"].birds_x, sub_target[sub_target.birds_x == "nocall"].birds_y)])
print(df_metrics.mean())


display(sub_target[sub_target.birds_x == "nocall"])